In [54]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor

# Load data
df = pd.read_csv(r"..\data\processed\top_15_items_df.csv")
df['date'] = pd.to_datetime(df['date'])



C:\Users\anush\AppData\Local\Temp\ipykernel_2204\3833463613.py:9: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"..\data\processed\top_15_items_df.csv")


In [55]:
df

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,total_revenue
0,FOODS_1_096_CA_1_evaluation,FOODS_1_096,FOODS_1,FOODS,CA_1,CA,d_1,0,2011-01-29,11101,...,2011,NaN,NaN,NaN,NaN,0,0,0,6.22,0.000000
1,FOODS_3_090_CA_1_evaluation,FOODS_3_090,FOODS_3,FOODS,CA_1,CA,d_1,107,2011-01-29,11101,...,2011,NaN,NaN,NaN,NaN,0,0,0,1.25,133.750000
2,FOODS_3_202_CA_1_evaluation,FOODS_3_202,FOODS_3,FOODS,CA_1,CA,d_1,9,2011-01-29,11101,...,2011,NaN,NaN,NaN,NaN,0,0,0,4.38,39.410156
3,FOODS_3_226_CA_1_evaluation,FOODS_3_226,FOODS_3,FOODS,CA_1,CA,d_1,13,2011-01-29,11101,...,2011,NaN,NaN,NaN,NaN,0,0,0,1.48,19.246094
4,FOODS_3_252_CA_1_evaluation,FOODS_3_252,FOODS_3,FOODS,CA_1,CA,d_1,19,2011-01-29,11101,...,2011,NaN,NaN,NaN,NaN,0,0,0,1.48,28.128906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260380,FOODS_3_555_WI_3_evaluation,FOODS_3_555,FOODS_3,FOODS,WI_3,WI,d_1941,39,2016-05-22,11617,...,2016,NaN,NaN,NaN,NaN,0,0,0,1.68,65.507810
260381,FOODS_3_586_WI_3_evaluation,FOODS_3_586,FOODS_3,FOODS,WI_3,WI,d_1941,37,2016-05-22,11617,...,2016,NaN,NaN,NaN,NaN,0,0,0,1.68,62.148438
260382,FOODS_3_587_WI_3_evaluation,FOODS_3_587,FOODS_3,FOODS,WI_3,WI,d_1941,24,2016-05-22,11617,...,2016,NaN,NaN,NaN,NaN,0,0,0,2.48,59.531250
260383,FOODS_3_694_WI_3_evaluation,FOODS_3_694,FOODS_3,FOODS,WI_3,WI,d_1941,48,2016-05-22,11617,...,2016,NaN,NaN,NaN,NaN,0,0,0,1.68,80.625000


In [44]:
# Ensure 'date' is in datetime format
df["date"] = pd.to_datetime(df["date"])

# Filter for the specific date
target_df = df[df["date"] == "2016-05-22"]

# Group by id, date, and state_id, and sum the revenue
grouped = target_df.groupby(["id", "date", "store_id"])["total_revenue"].sum().reset_index()

# Display result
print(grouped)

                                  id       date store_id  total_revenue
0        FOODS_1_096_CA_1_evaluation 2016-05-22     CA_1       0.000000
1        FOODS_1_096_CA_2_evaluation 2016-05-22     CA_2      17.214844
2        FOODS_1_096_CA_3_evaluation 2016-05-22     CA_3      11.476562
3        FOODS_1_096_CA_4_evaluation 2016-05-22     CA_4       5.738281
4        FOODS_1_096_TX_1_evaluation 2016-05-22     TX_1      44.652344
..                               ...        ...      ...            ...
145  HOUSEHOLD_1_535_TX_2_evaluation 2016-05-22     TX_2      34.843750
146  HOUSEHOLD_1_535_TX_3_evaluation 2016-05-22     TX_3      55.750000
147  HOUSEHOLD_1_535_WI_1_evaluation 2016-05-22     WI_1      20.906250
148  HOUSEHOLD_1_535_WI_2_evaluation 2016-05-22     WI_2       0.000000
149  HOUSEHOLD_1_535_WI_3_evaluation 2016-05-22     WI_3      13.937500

[150 rows x 4 columns]


In [45]:


# Make sure date column is in datetime format
df['date'] = pd.to_datetime(df['date'])

# Step 1: Find the latest date
latest_date = df['date'].max()

# Step 2: Filter for rows matching the latest date
latest_df = df[df['date'] == latest_date]

# Step 3: Group by item_id and date to get revenue summary
revenue_summary = (
    latest_df.groupby(['item_id', 'date'])['total_revenue']
    .sum()
    .reset_index()
)

# Step 4: Print the DataFrame directly
print(revenue_summary)


            item_id       date  total_revenue
0       FOODS_1_096 2016-05-22     808.074214
1       FOODS_3_090 2016-05-22    1035.701166
2       FOODS_3_120 2016-05-22    1773.046880
3       FOODS_3_202 2016-05-22    1138.105474
4       FOODS_3_226 2016-05-22     428.320312
5       FOODS_3_252 2016-05-22     657.312499
6       FOODS_3_444 2016-05-22    1528.722656
7       FOODS_3_555 2016-05-22     537.499996
8       FOODS_3_586 2016-05-22     979.257814
9       FOODS_3_587 2016-05-22     729.257821
10      FOODS_3_694 2016-05-22     428.320310
11      FOODS_3_714 2016-05-22     409.240227
12    HOBBIES_1_158 2016-05-22    1149.218745
13    HOBBIES_1_354 2016-05-22    2252.468750
14  HOUSEHOLD_1_535 2016-05-22     459.937500


In [ ]:
# Step 5: Get the total revenue across all items on the latest date
total_revenue_for_latest_day = revenue_summary['total_revenue'].sum()

print(f"Grand total revenue on {latest_date.date()} is {total_revenue_for_latest_day:.2f}")

Grand total revenue on 2016-05-22 is 14314.48


In [47]:
df.isnull().sum()

id                    0
item_id               0
dept_id               0
cat_id                0
store_id              0
state_id              0
d                     0
sales                 0
date                  0
wm_yr_wk              0
weekday               0
wday                  0
month                 0
year                  0
event_name_1     239191
event_type_1     239191
event_name_2     259854
event_type_2     259854
snap_CA               0
snap_TX               0
snap_WI               0
sell_price            0
total_revenue         0
dtype: int64

In [48]:
df.describe(include='all')

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,total_revenue
count,260385,260385,260385,260385,260385,260385,260385,260385.000000,260385,260385.000000,...,260385.000000,21194,21194,531,531,260385.000000,260385.000000,260385.000000,260385.000000,260385.000000
unique,150,15,4,3,10,3,1941,NaN,NaN,NaN,...,NaN,30,4,4,2,NaN,NaN,NaN,NaN,NaN
top,FOODS_1_096_CA_1_evaluation,FOODS_1_096,FOODS_3,FOODS,CA_3,CA,d_1907,NaN,NaN,NaN,...,NaN,Mother's day,Religious,Father's day,Cultural,NaN,NaN,NaN,NaN,NaN
freq,1941,19410,205299,224709,26378,105169,150,NaN,NaN,NaN,...,NaN,813,7239,146,386,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.220147,2013-11-22 19:01:56.094244608,11359.175605,...,2013.410181,NaN,NaN,NaN,NaN,0.329424,0.329677,0.329681,4.427623,51.231235
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,2011-01-29 00:00:00,11101.000000,...,2011.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.990000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,2012-08-15 00:00:00,11229.000000,...,2012.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,1.580000,10.078125
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.000000,2013-12-24 00:00:00,11348.000000,...,2013.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,1.680000,31.914062
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.000000,2015-03-16 00:00:00,11507.000000,...,2015.000000,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000,5.970000,68.500000
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,763.000000,2016-05-22 00:00:00,11617.000000,...,2016.000000,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000,23.980000,1618.734400


In [49]:

# Create calendar features
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day"] = df["date"].dt.day
df["weekday"] = df["date"].dt.weekday
df["wday"] = df["date"].dt.isocalendar().day
df["quarter"] = (df["month"] - 1) // 3 + 1

# Label encode categorical columns
cat_columns = ["item_id", "dept_id", "cat_id", "store_id", "state_id",
               "event_name_1", "event_type_1", "event_name_2", "event_type_2"]

encoders = {}
for col in cat_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    encoders[col] = le

# Save encoders
#joblib.dump(encoders, "../model/label_encoders.joblib")

# Features and target
features = ['sales', 'cat_id', 'sell_price', 'item_id', 'dept_id', 'store_id',
            'state_id', 'wm_yr_wk', 'wday', 'month', 'weekday', 'year',
            'snap_WI', 'snap_TX', 'event_type_1', 'snap_CA', 'event_name_1',
            'event_name_2', 'event_type_2', 'quarter']



In [50]:
df

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,total_revenue,day,quarter
0,FOODS_1_096_CA_1_evaluation,0,0,0,0,0,d_1,0,2011-01-29,11101,...,4,4,2,0,0,0,6.22,0.000000,29,1
1,FOODS_3_090_CA_1_evaluation,1,1,0,0,0,d_1,107,2011-01-29,11101,...,4,4,2,0,0,0,1.25,133.750000,29,1
2,FOODS_3_202_CA_1_evaluation,3,1,0,0,0,d_1,9,2011-01-29,11101,...,4,4,2,0,0,0,4.38,39.410156,29,1
3,FOODS_3_226_CA_1_evaluation,4,1,0,0,0,d_1,13,2011-01-29,11101,...,4,4,2,0,0,0,1.48,19.246094,29,1
4,FOODS_3_252_CA_1_evaluation,5,1,0,0,0,d_1,19,2011-01-29,11101,...,4,4,2,0,0,0,1.48,28.128906,29,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260380,FOODS_3_555_WI_3_evaluation,7,1,0,9,2,d_1941,39,2016-05-22,11617,...,4,4,2,0,0,0,1.68,65.507810,22,2
260381,FOODS_3_586_WI_3_evaluation,8,1,0,9,2,d_1941,37,2016-05-22,11617,...,4,4,2,0,0,0,1.68,62.148438,22,2
260382,FOODS_3_587_WI_3_evaluation,9,1,0,9,2,d_1941,24,2016-05-22,11617,...,4,4,2,0,0,0,2.48,59.531250,22,2
260383,FOODS_3_694_WI_3_evaluation,10,1,0,9,2,d_1941,48,2016-05-22,11617,...,4,4,2,0,0,0,1.68,80.625000,22,2


In [51]:
X = df[features]
y = df['total_revenue']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = XGBRegressor(n_estimators=100, max_depth=4, learning_rate=0.1)
model.fit(X_train, y_train)



,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [52]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Predict on training set
y_train_pred = model.predict(X_train)

# Calculate RMSE and R2 for training set
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_r2 = r2_score(y_train, y_train_pred)

# Calculate RMSE and R2 for test set
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

# Print results
print(f"Train RMSE: {train_rmse:.2f}, Test RMSE: {rmse:.2f}")
print(f"Train R2: {train_r2:.4f}, Test R2: {r2:.4f}")



Train RMSE: 5.09, Test RMSE: 5.75
Train R2: 0.9945, Test R2: 0.9929


In [53]:
'''import joblib

joblib.dump(model, "../model/xgb_regressor.joblib")'''

'import joblib\n\njoblib.dump(model, "../model/xgb_regressor.joblib")'